In [1]:
import baostock as bs
import pandas as pd
from tqdm import tqdm

# 基础配置：年度范围+季度范围
START_YEAR = 2013 
END_YEAR   = 2025        
QUARTERS = [1, 2, 3, 4]  # 新增：遍历4个季度

OUTPUT_CSV = "hs300_financial_quarterly_raw.csv"  # 重命名区分季度数据
OUTPUT_PKL = "hs300_financial_quarterly_raw.pkl"

lg = bs.login()

def get_hs300_codes(date="2024-12-31"):
    rs = bs.query_hs300_stocks(date=date)
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    df = pd.DataFrame(data_list, columns=rs.fields)
    return df['code'].tolist()

codes = get_hs300_codes()

# 函数保留quarter参数，不再默认固定为4
def fetch_profit_data(code, year, quarter):
    rs = bs.query_profit_data(code=code, year=year, quarter=quarter)
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    if not data_list:
        return None
    df = pd.DataFrame(data_list, columns=rs.fields)
    return df.iloc[0]

def fetch_balance_data(code, year, quarter):
    rs = bs.query_balance_data(code=code, year=year, quarter=quarter)
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    if not data_list:
        return None
    df = pd.DataFrame(data_list, columns=rs.fields)
    return df.iloc[0]

records = []
years = list(range(START_YEAR, END_YEAR + 1))
# 新增：遍历每个季度（核心修改）
for code in tqdm(codes, desc="Downloading HS300 quarterly financial data"):
    for year in years:
        for quarter in QUARTERS:  # 循环Q1-Q4
            prof_row = fetch_profit_data(code, year, quarter)
            bal_row  = fetch_balance_data(code, year, quarter)

            if prof_row is None or bal_row is None:
                continue

            try:
                rec = {
                    "code": code,
                    "year": year,
                    "quarter": quarter,  # 新增：记录季度
                    "roeAvg":        float(prof_row["roeAvg"]) if prof_row["roeAvg"] else None,
                    "npMargin":      float(prof_row["npMargin"]) if prof_row["npMargin"] else None,
                    "gpMargin":      float(prof_row["gpMargin"]) if prof_row["gpMargin"] else None,
                    "netProfit":     float(prof_row["netProfit"]) if prof_row["netProfit"] else None,
                    "epsTTM":        float(prof_row["epsTTM"]) if prof_row["epsTTM"] else None,
                    "MBRevenue":     float(prof_row["MBRevenue"]) if prof_row["MBRevenue"] else None,
                    "liabilityToAsset": float(bal_row["liabilityToAsset"]) if bal_row["liabilityToAsset"] else None,
                }
                records.append(rec)
            except Exception as e:
                continue

fin_df = pd.DataFrame(records)
bs.logout()

fin_df.to_csv(OUTPUT_CSV, index=False)
fin_df.to_pickle(OUTPUT_PKL)

print("已保存季度财务数据：")
print(" -", OUTPUT_CSV)
print(" -", OUTPUT_PKL)

login success!


logout success!


OSError: Cannot save file into a non-existent directory: 'data'

In [1]:
import os
import baostock as bs
import pandas as pd
from tqdm import tqdm

# === 你已经下载好的 base（profit+balance）===
BASE_PKL = "hs300_financial_quarterly_raw.pkl"

# === 新增数据（profit额外字段 + cf/gr/op/dp）===
EXTRA_PKL = "hs300_financial_quarterly_extra.pkl"
EXTRA_CSV = "hs300_financial_quarterly_extra.csv"

# === 合并输出 ===
ALL_PKL = "hs300_financial_quarterly_all.pkl"
ALL_CSV = "hs300_financial_quarterly_all.csv"

KEY_COLS = ["code", "year", "quarter"]


def auto_cast(v):
    """空值->None；能转float就转；否则保留原字符串"""
    if v is None:
        return None
    if isinstance(v, str):
        v = v.strip()
        if v == "" or v.lower() == "none":
            return None
    try:
        return float(v)
    except Exception:
        return v


def to_float(v):
    """仅用于你明确想要float的字段"""
    if v is None:
        return None
    if isinstance(v, str):
        v = v.strip()
        if v == "" or v.lower() == "none":
            return None
    try:
        return float(v)
    except Exception:
        return None


def first_existing_value(row_dict, candidates):
    for k in candidates:
        if k in row_dict:
            v = row_dict.get(k)
            if v is None:
                return None
            if isinstance(v, str) and v.strip() == "":
                return None
            return v
    return None


def fetch_first_row_dict(rs):
    """通用：取第一行dict；无数据/报错返回 None"""
    if rs.error_code != "0":
        return None
    data_list = []
    while rs.next():
        data_list.append(rs.get_row_data())
    if not data_list:
        return None
    df = pd.DataFrame(data_list, columns=rs.fields)
    row = df.iloc[0].to_dict()
    return {k: auto_cast(v) for k, v in row.items()}


def build_key_set(df: pd.DataFrame):
    if df is None or df.empty:
        return set()
    return set(map(tuple, df[KEY_COLS].astype(str).values))


# === (1) 里提到的 profit 额外字段（候选字段名：避免版本差异）===
PROFIT_EXTRA_MAP = {
    "pubDate": ["pubDate", "publishDate"],
    "statDate": ["statDate", "reportDate", "statdate"],

    "operateProfit": ["operateProfit", "operatingProfit", "opProfit", "opPro"],
    "totalProfit": ["totalProfit", "profitTotal", "totProfit"],

    "netProfitExclExtra": [
        "netProfitExclExtra",
        "netProfitExcludingExtraordinary",
        "netProfitAfterNR",
        "netProfitExclNonRecurring",
        "netProfitExcludeExtraordinary",
    ],

    "epsBasic": ["epsBasic", "basicEPS", "eps_basic", "eps"],
    "epsDiluted": ["epsDiluted", "dilutedEPS", "eps_diluted"],

    "periodExpenseRate": ["periodExpenseRate", "expenseRate", "expenseToRevenue", "periodExpenseRatio"],
    "costRate": ["costRate", "costToRevenue", "costToIncome", "costRatio"],
}

# 1) 读 base
if not os.path.exists(BASE_PKL):
    raise FileNotFoundError(f"找不到 {BASE_PKL}，请先运行你的原始下载脚本生成它。")

base_df = pd.read_pickle(BASE_PKL).copy()
for c in KEY_COLS:
    base_df[c] = base_df[c].astype(str)

base_keys = build_key_set(base_df)

# 2) 读 extra（断点续跑）
if os.path.exists(EXTRA_PKL):
    extra_df = pd.read_pickle(EXTRA_PKL).copy()
    for c in KEY_COLS:
        extra_df[c] = extra_df[c].astype(str)
else:
    extra_df = pd.DataFrame(columns=KEY_COLS)

extra_keys = build_key_set(extra_df)

todo_keys = list(base_keys - extra_keys)
todo_keys.sort()

print(f"[INFO] base keys: {len(base_keys)}")
print(f"[INFO] extra keys already downloaded: {len(extra_keys)}")
print(f"[INFO] todo (new downloads): {len(todo_keys)}")

# 3) 下载新增项：profit额外字段 + cf/gr/op/dp 全部
lg = bs.login()
if lg.error_code != "0":
    raise RuntimeError(f"baostock login failed: {lg.error_msg}")

new_records = []
save_every = 300  # 断点保存频率

for (code, year, quarter) in tqdm(todo_keys, desc="Downloading PROF+CF/GR/OP/DP extra"):
    y = int(year)
    q = int(quarter)

    # ---- 4张“额外表” ----
    cf = fetch_first_row_dict(bs.query_cash_flow_data(code=code, year=y, quarter=q))
    gr = fetch_first_row_dict(bs.query_growth_data(code=code, year=y, quarter=q))
    op = fetch_first_row_dict(bs.query_operation_data(code=code, year=y, quarter=q))
    dp = fetch_first_row_dict(bs.query_dupont_data(code=code, year=y, quarter=q))

    # ---- profit（为了拿(1)里提到的额外字段）----
    prof = fetch_first_row_dict(bs.query_profit_data(code=code, year=y, quarter=q))

    rec = {"code": code, "year": year, "quarter": quarter}

    # (1) profit 额外字段：找得到就填，找不到 None
    if prof is None:
        for out_k in PROFIT_EXTRA_MAP.keys():
            rec[out_k] = None
    else:
        for out_k, candidates in PROFIT_EXTRA_MAP.items():
            v = first_existing_value(prof, candidates)
            if out_k in ("pubDate", "statDate"):
                rec[out_k] = v  # 日期字符串
            else:
                rec[out_k] = to_float(v)

    # cf/gr/op/dp：把返回的字段都带前缀存下来（避免重名）
    if cf:
        rec.update({f"cf_{k}": v for k, v in cf.items() if k not in KEY_COLS})
    if gr:
        rec.update({f"gr_{k}": v for k, v in gr.items() if k not in KEY_COLS})
    if op:
        rec.update({f"op_{k}": v for k, v in op.items() if k not in KEY_COLS})
    if dp:
        rec.update({f"dp_{k}": v for k, v in dp.items() if k not in KEY_COLS})

    new_records.append(rec)

    # 断点保存，避免中途挂掉
    if len(new_records) >= save_every:
        tmp = pd.DataFrame(new_records)
        extra_df = pd.concat([extra_df, tmp], ignore_index=True)
        extra_df.drop_duplicates(subset=KEY_COLS, keep="last", inplace=True)
        extra_df.to_pickle(EXTRA_PKL)
        extra_df.to_csv(EXTRA_CSV, index=False)
        new_records = []

# 收尾保存
if new_records:
    tmp = pd.DataFrame(new_records)
    extra_df = pd.concat([extra_df, tmp], ignore_index=True)
    extra_df.drop_duplicates(subset=KEY_COLS, keep="last", inplace=True)

extra_df.to_pickle(EXTRA_PKL)
extra_df.to_csv(EXTRA_CSV, index=False)

bs.logout()

# 4) 合并输出（base + extra）
all_df = base_df.merge(extra_df, on=KEY_COLS, how="left")
all_df.to_pickle(ALL_PKL)
all_df.to_csv(ALL_CSV, index=False)

print("新增字段（profit额外 + cf/gr/op/dp）已保存：")
print(" -", EXTRA_PKL)
print(" -", EXTRA_CSV)
print("合并后的全量数据已保存：")
print(" -", ALL_PKL)
print(" -", ALL_CSV)


[INFO] base keys: 13296
[INFO] extra keys already downloaded: 0
[INFO] todo (new downloads): 13296
login success!


logout success!
新增字段（profit额外 + cf/gr/op/dp）已保存：
 - hs300_financial_quarterly_extra.pkl
 - hs300_financial_quarterly_extra.csv
合并后的全量数据已保存：
 - hs300_financial_quarterly_all.pkl
 - hs300_financial_quarterly_all.csv
